In [1]:
# https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-12-31

    
# https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-12-31&start=51&ref_=adv_nxt
    
# https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-12-31&start=101&ref_=adv_nxt    

In [1]:
from requests import get
url = 'https://www.imdb.com/search/title/?title_type=feature&release_date=2018-01-01,2018-12-31&start=51&ref_=adv_nxt'
response = get(url)
print(response.text[:500])




<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         
        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">

    <meta name="apple-itunes-app" content="app-id=342792525, app-argument=imdb:///?src=mdot">



        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle"


In [2]:
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)



bs4.BeautifulSoup

In [3]:
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))

<class 'bs4.element.ResultSet'>
50


In [4]:
container = movie_containers[0]
first_movie = movie_containers[0]

In [6]:
def names(container):
    name_list = []
    for elem in container(text=re.compile(r'Director')):
        if elem:
            name_list = elem.parent.text

            names = name_list.split('\n')
            names = [i for i in names if i] 
        
        
            names_s = []
            for name in names: 
                name_s = name.strip()
                name_s = name_s.strip(",")
                names_s.append(name_s)  
                names_s = [i for i in names_s if "|" not in i] 

            star_ind = names_s.index('Stars:')
            if star_ind:
                stars = names_s[star_ind + 1:len(names_s)]
            
                if names_s[0][0:8] == 'Director':
                    director_s = names_s[1:star_ind ]

    
        
    return director_s, stars    



In [7]:
def vote_gross(container):
    gross_tot = None
    votes = container.find('p', class_ = 'sort-num_votes-visible')
    if votes:
        votes = votes.text
        votes = votes.replace('\n', ' ')
        votes = votes.replace('|', '')
        votes = votes.strip(" ")
        votes = votes.split(" ")
        votes = [i for i in votes if i]
        if votes[0] == 'Votes:':
                vote_s = votes[1]
        if len(votes) > 2:
            if votes[2] == 'Gross:':
                gross_tot = votes[3]
         
    return  vote_s, gross_tot  

In [8]:
import pandas as pd
data = []
# for container in movie_containers:

title = container.h3.a
if title:
    data.append(title.text)
else:
    data.append(None)

list_number = container.find('span', class_ = 'lister-item-index unbold text-primary')
if list_number:
    data.append(int(list_number.text.strip(".")))   
else:
    data.append(None)

year = container.h3.find('span', class_ = 'lister-item-year text-muted unbold')
if year:

    data.append(year.text.strip("()"))
else:
    data.append(None)

runtime = container.find('span', class_ = 'runtime')
if runtime:
    data.append(runtime.text.strip("()"))
else:
    data.append(None)

imdb_sc = container.strong
if imdb_sc:
    data.append(float(imdb_sc.text.strip("()")))
else:
    data.append(None)

mscore = container.find('span', class_ = 'metascore favorable')
if mscore:
    data.append(int(mscore.text))
else:
    data.append(None)                

genres = container.find('span', class_ = 'genre')
if genres:
    data.append(genres.text.strip("\n"))
else:
    data.append(None)                

MPAA_rating = container.find('span', class_ = 'certificate')
if MPAA_rating:
    data.append(MPAA_rating.text)
else:
    data.append(None)    


directors, stars = names(container)
if directors:
    data.append(directors)
else:
    data.append(None)

if stars:
    data.append(stars)
else:
    data.append(None)                


votes, gross = vote_gross(container)

if votes:
    data.append(votes)            
else:
    data.append(None)

if gross:
    data.append(gross)                
else:
    data.append(None)


df2 =  pd.DataFrame([data], columns = ['Film title', 'Imdb 2018 list number', 'Year released', 'Runtime', 'Imdb Score', 'Metascore', 'Genres', 'MPAA rating', 'Director(s)', 'Actor(s)', 'Votes', 'Gross'])              


    
    

NameError: name 're' is not defined

In [254]:
df2

,Film title,Imdb 2018 list number,Year released,Runtime,Imdb Score,Metascore,Genres,MPAA rating,Director(s),Actor(s),Votes,Gross
0,The Ballad of Buster Scruggs,51,2018,133 min,7.3,79,"Comedy, Drama, Musical",R,"[Ethan Coen, Joel Coen]","[Tim Blake Nelson, Willie Watson, Clancy Brown...","93,830",None


In [12]:
year = first_movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.strip("()")
int(year) 

2018

In [8]:
first_movie.h3.a.text

'The Ballad of Buster Scruggs'

In [9]:
first_movie_runtime = first_movie.find('span', class_ = 'runtime').text
first_movie_runtime

'133 min'

In [243]:
first_movie.strong
first_imdb = float(first_movie.strong.text)
type(first_imdb)

float

In [244]:
first_mscore = first_movie.find('span', class_ = 'metascore favorable')
first_mscore = int(first_mscore.text)
print(first_mscore)

79


In [12]:
first_mgenre = first_movie.find('span', class_ = 'genre')
genres = first_mgenre.text
genres = genres.strip("\n")
type(genres)

str

In [81]:
votes = container.find('span', attrs = {'name':'nv'})['data-value']
# first_votes_n = first_votes['data-value']
# first_votes
# int(first_votes_n)

NameError: name 'container' is not defined

In [82]:
first_MPAA_rating = first_movie.find('span', class_ = 'certificate')
first_MPAA_rating.text

'R'

In [15]:
first_gross = first_movie.find('span', attrs = {'name':'nv'})

In [16]:
second_movie = movie_containers[1]

In [83]:
second_nvs = second_movie.find_all('span', attrs = {'name':'nv'}) 
second_nvs

[<span data-value="63258" name="nv">63,258</span>,
 <span data-value="21,704,844" name="nv">$21.70M</span>]

In [18]:
second_nvs[1]['data-value']

'21,704,844'

In [19]:
# <span class="lister-item-index unbold text-primary">51.</span>

In [237]:
first_listnumber = first_movie.find('span', class_ = 'lister-item-index unbold text-primary')

In [242]:
int(first_listnumber.text.strip("."))
#<p class="">

51

In [22]:
import re
first_movie.find_all("a", {"href" : re.compile('/name/.*')})

[<a href="/name/nm0001053/">Ethan Coen</a>,
 <a href="/name/nm0001054/">Joel Coen</a>,
 <a href="/name/nm0625789/">Tim Blake Nelson</a>,
 <a href="/name/nm0914929/">Willie Watson</a>,
 <a href="/name/nm0000317/">Clancy Brown</a>,
 <a href="/name/nm1180243/">Danny McCarthy</a>]

In [185]:
name_list = []
for elem in first_movie(text=re.compile(r'Director')):
    name_list = elem.parent
names = name_list.text

In [186]:
names = names.split('\n')

In [44]:
for index, value in enumerate(names): 
    print(index, value)


0 
1     Directors:
2 Ethan Coen, 
3 Joel Coen
4 | 
5     Stars:
6 Tim Blake Nelson, 
7 Willie Watson, 
8 Clancy Brown, 
9 Danny McCarthy
10 


In [187]:
names_s = []
for name in names: 
    name_s = name.strip()
    name_s = name_s.strip(",")
    names_s.append(name_s)

In [201]:


star_ind = names_s.index('Stars:')
if names_s[0][0:8] == 'Director':
    director_s = names_s[1:star_ind ]
director_s

['Ethan Coen', 'Joel Coen']

In [199]:
stars = names_s[star_ind + 1:len(names_s)]
stars

['Tim Blake Nelson', 'Willie Watson', 'Clancy Brown', 'Danny McCarthy']

In [ ]:
names_s = [i for i in names_s if i] names_s = [i for i in names_s if i ! "|"]

names_s = [i for i in names_s if i] 
names_s = [i for i in names_s if i !  "|"] 
names_s

In [180]:
for elem in container(text=re.compile(r'Votes')):
    votes = elem.parent

In [ ]:
<p class="sort-num_votes-visible">
                <span class="text-muted">Votes:</span>
                <span name="nv" data-value="63226">63,226</span>
    <span class="ghost">|</span>                <span class="text-muted">Gross:</span>
                <span name="nv" data-value="21,704,844">$21.70M</span>
        </p>

In [225]:
votes = first_movie.find('p', class_ = 'sort-num_votes-visible').text
votes

'\nVotes:\n93,830\n'

In [228]:


votes = votes.replace('\n', ' ')
votes = votes.replace('|', '')
votes = votes.strip(" ")
votes = votes.split(" ")
# votes = votes[0].split(" ")

# gross = votes[1].split(" ")

In [229]:
votes

['Votes:', '93,830']

In [183]:
votes_s

['Votes:', '63,258']

In [184]:
gross

['Gross:', '$21.70M']

In [176]:
if vote[0] == 'Votes:':
    vote_s = vote[1]

In [177]:
vote_s

'63,258'

In [178]:
if gross[0] == 'Gross:':
    gross_tot = gross[1]

In [231]:
votes2 = second_movie.find('p', class_ = 'sort-num_votes-visible').text
votes2
votes2 = votes2.replace('\n', ' ')
votes2 = votes2.replace('|', '')
votes2 = votes2.strip(" ")
votes2 = votes2.split(" ")

In [236]:
votes2 = [i for i in votes2 if i]
len(votes2)

4

In [245]:
x = ['Film title', 'Imdb 2018 list number', 'Year released', 
     'Runtime', 'Imdb Score', 'Metascore', 'Genres', 'MPAA rating', 'Director(s)', 'Actor(s)'] 
len(x)

10

In [257]:
11827 // 50

236

In [258]:
236 * 50

11800